In [1]:
%matplotlib inline

from utils import *

# Estimating traffic

In [19]:
Traf     = pandas.concat(map( lambda name : pandas.read_hdf(
           name,
    key  = 'table')
           ,H5('traffic')
           )).query(boundaryQ).reset_index(drop=True)

In [20]:
lLobj    =  ['pickup_latitude','pickup_longitude']
Traf     = Traf.round({x:1 for x in lLobj}).groupby(
            lLobj
            ).key.sum().to_frame().assign(
 traffic = lambda DF : (DF.key-DF.key.min())/(DF.key.max()-DF.key.min())
            ).reset_index().rename(
 columns = {'key':'traffic_count'})
Traf.head()

,pickup_latitude,pickup_longitude,traffic_count,traffic
0,39.0,-74.0,1,0.0
1,39.0,-73.7,1,0.0
2,39.0,-73.4,1,0.0
3,39.0,-72.7,1,0.0
4,39.1,-74.4,1,0.0


#  population density

In [15]:
towns = pandas.read_csv('./data/us.csv').query('state_id=="NY"'
                                      ).drop_duplicates('zips').groupby(['lat','lng']).sum().reset_index()[['density','lat','lng']]
towns.head()

,density,lat,lng
0,3155.0,40.5834,-74.1496
1,1485.4,40.5876,-73.7092
2,5883.0,40.5887,-73.6660
3,1691.0,40.5894,-73.7296
4,796.9,40.5903,-73.5795


In [106]:
Z = geopandas.read_file(path('','road.geojson'))

In [111]:
Z.head()

,rw_type,rwjurisdic,continuous,created_by,l_low_hn,modified_b,bphys_id,rsubsect,collection,twisted_pa,...,boroughcod,posted_spe,trafdir,l_blockfac,physicalid,borough_in,accessible,truck_rout,l_high_hn,geometry
0,1,None,None,DCP,25,csarkissian,0,2K,None,N,...,3,0,TF,0,37785,None,None,None,43,LINESTRING (-73.98027180261785 40.685958254199...
1,2,None,None,CSCL,None,BlockfaceID_LOAD,0,1D,None,N,...,4,0,FT,92260988,7141,None,None,None,None,LINESTRING (-73.84554250435114 40.739657802135...
2,1,None,None,DCP,349,csarkissian,0,3A,None,N,...,3,0,TW,0,38225,None,None,None,393,LINESTRING (-73.96582204725706 40.688209422824...
3,1,None,None,O_Ortiz,134-001,BlockfaceID_LOAD,0,1C,None,N,...,4,0,TF,12261266,166986,None,None,None,134-099,LINESTRING (-73.85196038837651 40.574280057286...
4,1,None,None,DCP,51-001,BlockfaceID_LOAD,0,2C,None,N,...,4,0,TW,92263852,97055,None,None,None,51-099,LINESTRING (-73.86924838521583 40.740570212006...


# Roads

In [99]:
Z = geopandas.read_file(path('','road.geojson')).pipe(lambda DF:
 
      [pandas.DataFrame(
        {'length' : DF.geometry.length,
        'lng' : DF.geometry.centroid.x,
        'lat' : DF.geometry.centroid.y,}
      ),DF.geometry.bounds] )

roads = Z[0].append(
    Z[1][['miny','minx']].rename(columns={'miny':'lat','minx':'lng'}).assign(
    length = Z[0].length).append(
    Z[1][['maxy','maxx']].rename(columns={'maxy':'lat','maxx':'lng'}).assign(
    length = Z[0].length)
    )
    ).reset_index(drop=True)
roads.head()

,lat,length,lng
0,40.685641,0.000766,-73.980487
1,40.738772,0.001798,-73.845385
2,40.687477,0.001495,-73.965674
3,40.574114,0.000859,-73.852356
4,40.740227,0.000735,-73.869117


In [100]:
towns.drop_duplicates().shape

(1109, 3)

# AREA

In [78]:
areas = geopandas.read_file(path('','Community Districts.geojson')).pipe(lambda DF:
    pandas.DataFrame(
        {'area' : DF.geometry.area,
        'lng' : DF.geometry.centroid.x,
        'lat' : DF.geometry.centroid.y,}
    ))
areas

,area,lat,lng
0,0.001020,40.607216,-73.993636
1,0.000871,40.580649,-73.982140
2,0.000984,40.631227,-73.983260
3,0.000561,40.694549,-73.916836
4,0.000423,40.849604,-73.887530
5,0.000914,40.890504,-73.906298
6,0.001538,40.886299,-73.851497
7,0.000502,40.897060,-73.886611
8,0.000928,40.648043,-73.933817
9,0.000448,40.662800,-73.946927


# Merge all

In [101]:
preagged = [
    
    Link2DFS_by_nearestGeo(
	DF1= ref,
	DF2= towns,
	ColMapper=[['lat','lng']]*2,
	prefix='').pipe(
        lambda R : R.groupby(
            map('_{}'.format,towns.columns)
        ).agg({'lat':'count',('area','length')[i==1]:sum}

             ).rename(columns=lambda x : (('towns','roads')[i==1],('area','length')[i==1])[x != 'lat']
            ).reset_index()
    )
    
    for (ref,i) in zip([roads,areas],[1,2])
]



In [112]:
preagged[0]

,_density,_lat,_lng,roads,length
0,606.0,40.8162,-73.7408,156,0.200563
1,659.0,40.7680,-73.7089,828,1.010724
2,1471.0,40.7858,-73.7396,319,0.291845
3,1485.4,40.7881,-73.7492,2859,3.523571
4,1622.0,40.8930,-73.8057,841,1.309126
5,1691.0,40.5894,-73.7296,572,0.633749
6,1826.4,40.7752,-73.7280,2477,3.083789
7,2943.5,40.8884,-73.8282,6529,7.739773
8,2997.9,40.6557,-73.7185,7068,7.109060
9,3155.0,40.5834,-74.1496,48862,57.336230


In [103]:
roads.describe()

,lat,length,lng
count,351573.000000,351573.000000,351573.000000
mean,40.707151,0.001106,-73.919412
std,0.089747,0.001000,0.116616
min,40.497860,0.000002,-74.255209
25%,40.637066,0.000543,-73.979175
50%,40.707908,0.000885,-73.909854
75%,40.764558,0.001429,-73.836546
max,40.915103,0.041793,-73.700020


In [104]:
towns.describe()

,density,lat,lng
count,1109.000000,1109.000000,1109.000000
mean,897.925068,42.265927,-75.143420
std,1450.878247,1.125401,1.861779
min,0.000000,40.583400,-79.712300
25%,235.000000,41.142500,-76.370400
50%,482.100000,42.336300,-74.375500
75%,1058.600000,43.038400,-73.694200
max,27799.000000,44.993300,-71.944900


In [76]:
Link2DFS_by_nearestGeo(
	DF1= towns,
	DF2= Traf,
	ColMapper=[['lat','lng'],['pickup_latitude','pickup_longitude']],
	prefix='traffic')

,density,lat,lng,traffic_pickup_latitude,traffic_pickup_longitude,traffic_traffic_count,traffic_traffic,traffic_Dist
0,3155.0,40.5834,-74.1496,40.6,-74.1,2708,8.359698e-05,0.052304
1,1485.4,40.5876,-73.7092,40.6,-73.7,1209,3.730519e-05,0.015440
2,5883.0,40.5887,-73.6660,40.6,-73.7,1209,3.730519e-05,0.035829
3,1691.0,40.5894,-73.7296,40.6,-73.7,1209,3.730519e-05,0.031441
4,796.9,40.5903,-73.5795,40.6,-73.6,380,1.170419e-05,0.022679
5,1158.8,40.5904,-73.6121,40.6,-73.6,380,1.170419e-05,0.015446
6,3789.5,40.6019,-73.6647,40.6,-73.7,1209,3.730519e-05,0.035351
7,685.0,40.6042,-73.7149,40.6,-73.7,1209,3.730519e-05,0.015481
8,3391.2,40.6218,-73.7507,40.6,-73.8,816196,2.520555e-02,0.053905
9,952.0,40.6247,-73.6981,40.6,-73.7,1209,3.730519e-05,0.024773


In [8]:
result.drop(['id'],axis=1).to_hdf(
            path('','traffic_summary.h5'),
       key='table',
     index=None)